In [2]:
#Importing needed libs
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

# Bell State


In [3]:
entangled_00 = bell_state(state='00')
down = fock(2,0)
up = fock(2,1)
state_00 = tensor(down, down)
state_01 = tensor(down, up)
state_10 = tensor(up, down)
state_11 = tensor(up, up)


entangled_00

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]

#For Measuement

Tr(|0><0| * rhoAB.ptrace(0))

-> measures the probability of finding the first particle (rhoAB.ptrace(0)) in |0> state

Tr(|00><00| * rhoAB)

-> measures the probability of finding both particle in |0> state


In [4]:
#measuring

def get_probability(entanglement, answer, particle):
    entanglement = entanglement.unit()
    density_ab = entanglement * entanglement.dag()
    A = answer * answer.dag()
    B = A*density_ab.ptrace(particle)
    return B.tr()

def get_probability_both(entanglement, answer,answer2):
    entanglement = entanglement.unit()
    density_ab = entanglement * entanglement.dag()
    A = answer * answer.dag() 
    B = answer2*answer2.dag()
    A = tensor(A,B)
    C = A*density_ab
    return C.tr()

def print_all_single(entaglement):
    print("First particle being down is " + str(float(100 * get_probability(entaglement, down, 0))) + "%")
    print("First particle being up is " + str(float(100 * get_probability(entaglement, up, 0))) + "%")
    print("Second particle being down is " + str(float(100 * get_probability(entaglement, down, 1))) + "%")
    print("Second particle being up is " + str(float(100 * get_probability(entaglement, up, 1))) + "%")
    
def print_all_combined(entaglement):
    print("Particle being down down is " + str(float(100 * get_probability_both(entaglement, down, down))) + "%")
    print("Particle being down up is " + str(float(100 * get_probability_both(entaglement, down, up))) + "%")
    print("Particle being up down is " + str(float(100 * get_probability_both(entaglement, up, down))) + "%")
    print("Particle being up up is " + str(float(100 * get_probability_both(entaglement, up, up))) + "%")

def print_all_detail(entaglement, title):
    print()
    print("------------  " + title + " ----------------")
    print_all_single(entaglement)
    print("--------------------------------------------")
    print_all_combined(entaglement)
    print()
    
print_all_detail(entangled_00, "For Bell State 00")

entangled_01 = bell_state(state='01')
print_all_detail(entangled_01, "For Bell State 01")

entangled_10 = bell_state(state='10')
print_all_detail(entangled_10, "For Bell State 10")

entangled_11 = bell_state(state='11')
print_all_detail(entangled_11, "For Bell State 11")


------------  For Bell State 00 ----------------
First particle being down is 50.000000000000014%
First particle being up is 50.000000000000014%
Second particle being down is 50.000000000000014%
Second particle being up is 50.000000000000014%
--------------------------------------------
Particle being down down is 50.000000000000014%
Particle being down up is 0.0%
Particle being up down is 0.0%
Particle being up up is 50.000000000000014%


------------  For Bell State 01 ----------------
First particle being down is 50.000000000000014%
First particle being up is 50.000000000000014%
Second particle being down is 50.000000000000014%
Second particle being up is 50.000000000000014%
--------------------------------------------
Particle being down down is 50.000000000000014%
Particle being down up is 0.0%
Particle being up down is 0.0%
Particle being up up is 50.000000000000014%


------------  For Bell State 10 ----------------
First particle being down is 50.000000000000014%
First particl

# Partially entangled states


#For Creating mixed entangled states

entangled = x|00><00| + b|01><10| ...


In [5]:
entangled_mixed_clean1 = (np.sqrt(3)/2 * state_00 +  0.5 * state_11).unit() 
entangled_mixed_clean1


Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.8660254]
 [0.       ]
 [0.       ]
 [0.5      ]]

In [6]:
print_all_detail(entangled_mixed_clean1, "For a partially entangled state 1")

entangled_mixed_clean2 = (np.sqrt(3)/2 * state_01 -  0.5 * state_10).unit()
print_all_detail(entangled_mixed_clean2, "For a partially entangled state 2")

entangled_mixed_1 = (0.5 * state_01 -  2.2 * state_10).unit()
print_all_detail(entangled_mixed_1, "For a partially entangled state 3")

entangled_mixed_2 = (1.2 * state_00 -  0.6 * state_11).unit()
print_all_detail(entangled_mixed_2, "For a partially entangled state 4")



------------  For a partially entangled state 1 ----------------
First particle being down is 74.99999999999999%
First particle being up is 25.0%
Second particle being down is 74.99999999999999%
Second particle being up is 25.0%
--------------------------------------------
Particle being down down is 74.99999999999999%
Particle being down up is 0.0%
Particle being up down is 0.0%
Particle being up up is 25.0%


------------  For a partially entangled state 2 ----------------
First particle being down is 74.99999999999999%
First particle being up is 25.0%
Second particle being down is 25.0%
Second particle being up is 74.99999999999999%
--------------------------------------------
Particle being down down is 0.0%
Particle being down up is 74.99999999999999%
Particle being up down is 25.0%
Particle being up up is 0.0%


------------  For a partially entangled state 3 ----------------
First particle being down is 4.911591355599215%
First particle being up is 95.08840864440081%
Second par

# Entangled states with random errors

In [138]:
entangled_imperfect_1 = (0.7 * state_00 + 0.1 * state_01 + 0.1 * state_10 +   0.7 * state_11).unit() 
entangled_imperfect_1

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.7]
 [0.1]
 [0.1]
 [0.7]]

In [1]:
print_all_detail(entangled_imperfect_1, "For a entangled state with errors 1")

entangled_imperfect_2 = (0.1 * state_00 + 0.8 * state_01 + 0.8 * state_10 +   0.1 * state_11).unit()
print_all_detail(entangled_imperfect_2, "For a entangled state with errors 2")

NameError: name 'print_all_detail' is not defined

# Entangled states with time dependent coefficients

# Hamiltonian-based time-dependent states